### ***NEURAL STYLE TRANSFER***

In [ ]:
#import pretrain model
import os
import tensorflow as tf
import IPython.display as display
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import PIL.Image
import time
import functools
import cv2
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

In [ ]:
content = cv2.imread('/content/MonaLisa.jpg')
style = cv2.imread('/content/Van_Gohn.jpg')

In [ ]:
x = tf.keras.applications.vgg19.preprocess_input(content)
x = np.resize(x, [224, 224, 3])
x = tf.expand_dims(x, axis = 0)
vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')

574717952/574710816 [==============================] - 8s 0us/step


In [ ]:
for layer in vgg.layers:
  print(layer.name)

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool
flatten
fc1
fc2
predictions


In [ ]:
content_layers = ['block2_conv2']

style_layers = ['block1_conv1',
               'block2_conv1',
               'block3_conv1',
               'block4_conv1',
               'block5_conv1']
number_content_layer = len(content_layer)
number_style_layer = len(style_layer)

In [ ]:
def vgg_layer(layer_names):
  vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')
  vgg.trainable = False

  outputs = [vgg.get_layer(name).output for name in layer_names]

  model = tf.keras.Model(inputs=[vgg.input], outputs = outputs)

  return model


In [ ]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_location = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_location)


In [ ]:
class StyleContentModel(tf.keras.models.Model):
  def __init__(self, styles_layer, contents_layer):
    super(StyleContentModel).__init__()
    self.vgg = vgg_layer(styles_layer + contents_layer)
    self.styles_layer = styles_layer
    self.contents_layer = contents_layer
    self.num_styles_layer = len(styles_layer)
    self.contents_layer = contents_layer
    self.vgg.trainable = False

  def call(self, inputs):
    inputs = inputs*255
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_styles_layer],
                                      outputs[self.num_styles_layer:])
    
    style_outputs = [gram_matrix(style_output) for style_output in style_ouputs]

    content_dict = {content: value
                    for content_name, value
                    in zip(self.contents_layer, content_outputs)}

    style_dict = {style_name: value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}

    return {'content': content_dict, 'style': style_dict}

In [ ]:
style_targets = extractor(style)['style']
content_targets = extractor(content)['content']

In [ ]:
image = tf.Variable(content_image)

In [ ]:
def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

In [ ]:
opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

In [ ]:
style_weight=1e2
content_weight=1e5

In [ ]:
def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss

In [ ]:
@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

In [ ]:
image = tf.Variable(content_image)

In [ ]:
import time
start = time.time()

epochs = 10
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end='', flush=True)
  display.clear_output(wait=True)
  display.display(tensor_to_image(image))
  print("Train step: {}".format(step))

end = time.time()
print("Total time: {:.1f}".format(end-start))